In [1]:
from pathlib import Path
import os
def load_env_vars_from_file(path: Path):
    file = path.read_text()
    for line in file.splitlines():
        if line.startswith("export"):
            var_to_value = line.split(" ")[1].split("=")
            var_ = var_to_value[0]
            value_ = var_to_value[1][1:-1]
            os.environ[var_] = value_
def set_up_prod_credentials():
    file = "../../../../scripts/prod_setup_creds.sh"
    load_env_vars_from_file(Path(file))
set_up_prod_credentials()

In [2]:
from exchange.interface import ExchangeInterface

e = ExchangeInterface(is_test_run=False)

In [3]:
from datetime import datetime
from helpers.types.portfolio import GetFillsRequest
import pytz
start = datetime(2024, 10, 14).astimezone(pytz.UTC)
fills = e.get_fills(GetFillsRequest(min_ts=int(start.timestamp())))

In [4]:
fills.sort(key=lambda x: x.created_time)

In [11]:
from collections import defaultdict
from dataclasses import dataclass

from helpers.types.orders import Side, TradeType

@dataclass
class ProfitData():
    positions: int = 0 
    spent: int = 0 
    made: int = 0 
# From market ticker to position
pd = defaultdict(lambda: ProfitData())
# Had taker fills

for fill in fills:
    if fill.is_taker == True:
        print("taker fill: ", fill.ticker)

    if (fill.side == Side.YES and fill.action == TradeType.BUY) or (fill.side == Side.NO and fill.action == TradeType.SELL):
        pd[fill.ticker].spent += (fill.yes_price) * (fill.count)
        if pd[fill.ticker].positions < 0:
            pd[fill.ticker].made += min(fill.count, abs(pd[fill.ticker].positions))  * 100
        pd[fill.ticker].positions += fill.count
    else:
        pd[fill.ticker].spent += (fill.no_price) * (fill.count)
        if pd[fill.ticker].positions > 0:
            pd[fill.ticker].made += min(fill.count, abs(pd[fill.ticker].positions)) * 100
        pd[fill.ticker].positions -= fill.count



action=<TradeType.BUY: 'buy'> count=20 created_time=datetime.datetime(2024, 10, 15, 15, 44, 9, 380708, tzinfo=TzInfo(UTC)) is_taker=False no_price=67 order_id='ce100a3d-ccf4-4228-bfb1-2cf97ec88c6d' side=<Side.NO: 'no'> ticker='HIGHAUS-24OCT15-B96.5' trade_id='07e06ca1-ce8d-4a37-b851-c589f7d9d974' yes_price=33
ProfitData(positions=-20, spent=1340, made=0)
taker fill:  HIGHAUS-24OCT15-B96.5
action=<TradeType.BUY: 'buy'> count=40 created_time=datetime.datetime(2024, 10, 15, 15, 44, 28, 652606, tzinfo=TzInfo(UTC)) is_taker=True no_price=65 order_id='e57a5c61-0ff2-4dc9-b7af-706eb58f343f' side=<Side.YES: 'yes'> ticker='HIGHAUS-24OCT15-B96.5' trade_id='0f8c54c1-6fc1-4bfd-9f07-9259029ffd7c' yes_price=35
ProfitData(positions=20, spent=2740, made=2000)
action=<TradeType.BUY: 'buy'> count=15 created_time=datetime.datetime(2024, 10, 15, 15, 48, 31, 641392, tzinfo=TzInfo(UTC)) is_taker=False no_price=69 order_id='06c49c98-d734-46e1-a291-dab435eb9e7b' side=<Side.NO: 'no'> ticker='HIGHAUS-24OCT15-B96

In [7]:
pd

defaultdict(<function __main__.<lambda>()>,
            {'HIGHCHI-24OCT15-B50.5': ProfitData(positions=-7, spent=5442, made=9300),
             'HIGHNY-24OCT15-B58.5': ProfitData(positions=-20, spent=9148, made=12400),
             'HIGHCHI-24OCT15-B48.5': ProfitData(positions=-3, spent=2582, made=2700),
             'HIGHNY-24OCT15-B56.5': ProfitData(positions=0, spent=4305, made=6000),
             'HIGHMIA-24OCT15-B86.5': ProfitData(positions=-12, spent=9333, made=18100),
             'HIGHAUS-24OCT15-B96.5': ProfitData(positions=20, spent=5894, made=10500),
             'HIGHAUS-24OCT15-B98.5': ProfitData(positions=-20, spent=3710, made=8200),
             'INX-24OCT15-B5812': ProfitData(positions=-20, spent=1540, made=0),
             'HIGHMIA-24OCT15-B88.5': ProfitData(positions=20, spent=640, made=0)})

In [6]:

from helpers.types.markets import MarketResult


final_profit = dict()
for ticker, data in pd.items():
    profit = data.made - data.spent
    if data.positions != 0:
        try:
            m = e.get_market(ticker)
            fi = e.get_fills(GetFillsRequest(ticker=ticker))
        except Exception as ex:
            print(ex)
            continue
        fi.sort(key=lambda x: x.created_time)
        # Dont consider this market if there were fills before the start time 
        if len(fi) > 0 and fi[0].created_time < start:
            print('Had fills before start time: ', m.ticker)
            continue
        if m.result == MarketResult.NO:
            if data.positions < 0:
                profit += abs(data.positions) * 100
        elif m.result == MarketResult.YES:
            if data.positions > 0:
                profit += abs(data.positions) * 100
        else:
            # This market has not settled yet
            print('market not settled: ', m.ticker)
            continue 
    final_profit[ticker] = profit


In [9]:
final_profit

{'HIGHCHI-24OCT15-B50.5': 558,
 'HIGHNY-24OCT15-B58.5': 852,
 'HIGHCHI-24OCT15-B48.5': -382,
 'HIGHNY-24OCT15-B56.5': -405,
 'HIGHMIA-24OCT15-B86.5': -733,
 'HIGHAUS-24OCT15-B96.5': -194,
 'HIGHAUS-24OCT15-B98.5': 390,
 'INX-24OCT15-B5812': -1540,
 'HIGHMIA-24OCT15-B88.5': -640}

In [8]:
sorted(final_profit.items(), key=lambda x: x[1])

[('INX-24OCT15-B5812', -1540),
 ('HIGHMIA-24OCT15-B86.5', -733),
 ('HIGHMIA-24OCT15-B88.5', -640),
 ('HIGHNY-24OCT15-B56.5', -405),
 ('HIGHCHI-24OCT15-B48.5', -382),
 ('HIGHAUS-24OCT15-B96.5', -194),
 ('HIGHAUS-24OCT15-B98.5', 390),
 ('HIGHCHI-24OCT15-B50.5', 558),
 ('HIGHNY-24OCT15-B58.5', 852)]

In [12]:
sum(final_profit.values())

-2094